In [2]:
import sys
import numpy as np
sys.path.append("UF")
sys.path.append("..")
sys.path.append("../UF")
%load_ext autoreload
%autoreload 

In [3]:
from qiskit_qec.circuits import ArcCircuit
from qiskit_qec.decoders import UnionFindDecoder
from qiskit_aer import AerSimulator
from qiskit.providers.fake_provider import FakeSherbrooke
from qiskit import QuantumCircuit, QuantumRegister
import numpy as np

In [4]:
links = [[0, 1, 2], [2, 3, 4], [4, 5, 6], [6, 7, 8]]
T = 5 # number of syndrome measurement rounds
code = ArcCircuit(links, T, basis='zz',logical='1')
qc = code.circuit['zz']
qc.draw(fold=-1)

░      ┌───┐     ┌───┐                     ░ ┌─┐                     ░      ┌───┐     ┌───┐                     ░ ┌─┐                     ░      ┌───┐     ┌───┐                     ░ ┌─┐                     ░      ┌───┐     ┌───┐                     ░ ┌─┐                     ░      ┌───┐     ┌───┐                     ░ ┌─┐                        
      link_qubit_0: ──────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├─|0>─────────────────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├─|0>─────────────────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├─|0>─────────────────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├─|0>─────────────────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├────────────────────────
                          ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘┌─┐                     
      link_qubit_1: ──────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫─┤M├─────────────────────
                          ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║ └╥┘┌─┐                  
      link_qubit_2: ──────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫───╫───┤M├──|0>───────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫───╫───┤M├──|0>───────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫───╫───┤M├──|0>───────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫───╫───┤M├──|0>───────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫──╫─┤M├──────────────────
                          ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║   ║   └╥┘  ┌─┐       ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║   ║   └╥┘  ┌─┐       ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║   ║   └╥┘  ┌─┐       ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║   ║   └╥┘  ┌─┐       ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║  ║ └╥┘┌─┐               
      link_qubit_3: ──────░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫───╫────╫───┤M├──|0>──░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫───╫────╫───┤M├──|0>──░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫───╫────╫───┤M├──|0>──░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫───╫────╫───┤M├──|0>──░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫──╫──╫─┤M├───────────────
                    ┌───┐ ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║   ║    ║   └╥┘       ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║   ║    ║   └╥┘       ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║   ║    ║   └╥┘       ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║   ║    ║   └╥┘       ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║  ║  ║ └╥┘┌─┐            
      code_qubit_0: ┤ X ├─░───┼────■────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───┼────■────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───┼────■────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───┼────■────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───┼────■────┼────┼────┼────┼────┼────┼───░──╫──╫──╫──╫─┤M├────────────
                    ├───┤ ░   │         │    │    │    │    │    │   ░  ║   ║    ║    ║        ░   │         │    │    │    │    │    │   ░  ║   ║    ║    ║        ░   │         │    │    │    │    │    │   ░  ║   ║    ║    ║        ░   │         │    │    │    │    │    │   

In [5]:
counts = AerSimulator().run(qc).result().get_counts()
fake_counts = FakeSherbrooke().run(qc, shots=10000).result().get_counts()
fake_counts = sorted(fake_counts.items(), key=lambda x: x[1], reverse=True)
print(str(counts))
print(fake_counts)

{'11111 0000 0000 0000 0000 0000': 1024}
[('11111 0000 0000 0000 0000 0000', 5384), ('11011 0000 0000 0000 0000 0000', 606), ('11111 0000 0001 0000 0000 0000', 226), ('11111 0000 0000 0000 0001 0000', 215), ('11111 0000 0000 0001 0000 0000', 213), ('11111 0001 0000 0000 0000 0000', 213), ('11111 0000 0000 0000 0000 0001', 209), ('11111 0000 0000 0000 0000 0100', 158), ('11111 0100 0000 0000 0000 0000', 139), ('11111 0000 0100 0000 0000 0000', 136), ('11111 0000 0000 0000 0100 0000', 136), ('11111 0000 0000 0100 0000 0000', 131), ('11111 0000 0000 0000 0000 0010', 87), ('11111 0000 1000 0000 0000 0000', 85), ('11111 0000 0000 0010 0000 0000', 82), ('11111 0000 0010 0000 0000 0000', 76), ('11111 0000 0000 0000 0010 0000', 76), ('11101 0000 0000 0000 0000 0000', 74), ('11111 0000 0000 1000 0000 0000', 71), ('01111 0000 0000 0000 0000 0000', 68), ('11111 0000 0000 0000 0000 1000', 66), ('11111 0000 0000 0000 1000 0000', 64), ('11110 0000 0000 0000 0000 0000', 63), ('11111 1000 0000 0000 00

# Count order

In [6]:
insert_position = next((index for index, instr in enumerate(qc.data) if instr[0].name == 'barrier'), None) + 1
print("insert_position: ", insert_position)
print(qc.data)

insert_position:  6
[CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'code_qubit'), 0),), clbits=()), CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'code_qubit'), 1),), clbits=()), CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'code_qubit'), 2),), clbits=()), CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'code_qubit'), 3),), clbits=()), CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'code_qubit'), 4),), clbits=()), CircuitInstruction(operation=Instruction(name='barrier', num_qubits=9, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(4, 'link_qubit'), 0), Qubit(QuantumRegister(4, 'link_qubit'), 1), Q

In [7]:
new_qc = qc.copy()
new_qc.data.clear()
new_qc.data += qc.data[:insert_position]
new_qc.x(0)
new_qc.data += qc.data[insert_position:]
new_qc.draw(fold=-1)

░ ┌───┐     ┌───┐          ┌───┐           ░ ┌─┐                     ░      ┌───┐     ┌───┐                     ░ ┌─┐                     ░      ┌───┐     ┌───┐                     ░ ┌─┐                     ░      ┌───┐     ┌───┐                     ░ ┌─┐                     ░      ┌───┐     ┌───┐                     ░ ┌─┐                        
      link_qubit_0: ──────░─┤ X ├─────┤ X ├──────────┤ X ├───────────░─┤M├─|0>─────────────────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├─|0>─────────────────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├─|0>─────────────────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├─|0>─────────────────░──────┤ X ├─────┤ X ├─────────────────────░─┤M├────────────────────────
                          ░ └───┘     └─┬─┘          └─┬─┘┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘ ┌─┐                 ░      └─┬─┘     └─┬─┘          ┌───┐┌───┐ ░ └╥┘┌─┐                     
      link_qubit_1: ──────░─────────────┼──────────────┼──┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫──┤M├──|0>────────────░────────┼─────────┼────────────┤ X ├┤ X ├─░──╫─┤M├─────────────────────
                          ░      ┌───┐  │  ┌───┐       │  └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║  └╥┘  ┌─┐            ░        │  ┌───┐  │  ┌───┐     └─┬─┘└─┬─┘ ░  ║ └╥┘┌─┐                  
      link_qubit_2: ──────░──────┤ X ├──┼──┤ X ├───────┼────┼────┼───░──╫───╫───┤M├──|0>───────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫───╫───┤M├──|0>───────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫───╫───┤M├──|0>───────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫───╫───┤M├──|0>───────░────────┼──┤ X ├──┼──┤ X ├───────┼────┼───░──╫──╫─┤M├──────────────────
                          ░ ┌───┐└─┬─┘  │  └─┬─┘┌───┐  │    │    │   ░  ║   ║   └╥┘  ┌─┐       ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║   ║   └╥┘  ┌─┐       ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║   ║   └╥┘  ┌─┐       ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║   ║   └╥┘  ┌─┐       ░ ┌───┐  │  └─┬─┘  │  └─┬─┘┌───┐  │    │   ░  ║  ║ └╥┘┌─┐               
      link_qubit_3: ──────░─┤ X ├──┼────┼────┼──┤ X ├──┼────┼────┼───░──╫───╫────╫───┤M├──|0>──░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫───╫────╫───┤M├──|0>──░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫───╫────╫───┤M├──|0>──░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫───╫────╫───┤M├──|0>──░─┤ X ├──┼────┼────┼────┼──┤ X ├──┼────┼───░──╫──╫──╫─┤M├───────────────
                    ┌───┐ ░ └─┬─┘  │    │    │  └─┬─┘  │    │    │   ░  ║   ║    ║   └╥┘       ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║   ║    ║   └╥┘       ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║   ║    ║   └╥┘       ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║   ║    ║   └╥┘       ░ └─┬─┘  │    │    │    │  └─┬─┘  │    │   ░  ║  ║  ║ └╥┘┌─┐            
      code_qubit_0: ┤ X ├─░───┼────┼────■────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───┼────■────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───┼────■────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───┼────■────┼────┼────┼────┼────┼────┼───░──╫───╫────╫────╫────────░───┼────■────┼────┼────┼────┼────┼────┼───░──╫──╫──╫──╫─┤M├────────────
                    ├───┤ ░   │    │         │    │    │    │    │   ░  ║   ║    ║    ║        ░   │         │    │    │    │    │    │   ░  ║   ║    ║    ║        ░   │         │    │    │    │    │    │   ░  ║   ║    ║    ║        ░   │         │    │    │    │    │    │   

In [8]:
counts = AerSimulator().run(new_qc).result().get_counts()
fake_counts = FakeSherbrooke().run(new_qc, shots=100).result().get_counts()
fake_counts = sorted(fake_counts.items(), key=lambda x: x[1], reverse=True)
print(str(counts))
print(fake_counts)

{'11111 0000 0000 0000 0000 0001': 1024}
[('11111 0000 0000 0000 0000 0001', 62), ('11101 0000 0000 0000 0000 0001', 6), ('11011 0000 0000 0000 0000 0001', 6), ('11111 0000 0100 0000 0000 0001', 3), ('11111 0100 0000 0000 0000 0001', 2), ('11111 0000 0000 0001 0000 0001', 2), ('11111 0000 0001 0000 0000 0001', 2), ('11111 0000 0000 0000 0100 0001', 2), ('11111 0001 0000 0000 0000 0001', 1), ('11111 0010 0000 0000 0000 0001', 1), ('11111 0000 0000 0100 0000 0001', 1), ('11111 0000 0000 0000 0000 0101', 1), ('11111 0001 0000 0000 0010 0001', 1), ('11111 0000 0000 0000 0000 0000', 1), ('11111 0000 0000 0000 0000 1001', 1), ('11111 1000 0000 0000 0000 0011', 1), ('11111 0000 0010 0000 0000 0001', 1), ('11111 0000 1000 0000 0000 0001', 1), ('11111 0000 0000 0000 0001 0001', 1), ('11111 0000 0000 1000 0000 0001', 1), ('10111 1100 1100 1100 1100 1101', 1), ('11101 0000 0000 0000 0100 0001', 1), ('11111 0000 0000 0000 0010 0001', 1)]


# IQ Data Order

In [28]:
from qiskit_ibm_provider import IBMProvider
from qiskit import transpile
from probabilities_functions import get_counts

provider = IBMProvider()
backend = provider.get_backend('ibmq_jakarta')

links = [[0, 1, 2], [2, 3, 4]]
T = 3 # number of syndrome measurement rounds
code = ArcCircuit(links, T, basis='zz',logical='1')

qc = code.circuit['zz']

new_qc = qc.copy()
new_qc.data.clear()
new_qc.x(0)
new_qc.data += qc.data


qc = new_qc
qc.draw(fold=-1)


┌───┐ ░ ┌───┐┌───┐           ░ ┌─┐           ░ ┌───┐┌───┐           ░ ┌─┐           ░ ┌───┐┌───┐           ░ ┌─┐            
      link_qubit_0: ┤ X ├─░─┤ X ├┤ X ├───────────░─┤M├─|0>───────░─┤ X ├┤ X ├───────────░─┤M├─|0>───────░─┤ X ├┤ X ├───────────░─┤M├────────────
                    └───┘ ░ └─┬─┘└─┬─┘┌───┐┌───┐ ░ └╥┘ ┌─┐       ░ └─┬─┘└─┬─┘┌───┐┌───┐ ░ └╥┘ ┌─┐       ░ └─┬─┘└─┬─┘┌───┐┌───┐ ░ └╥┘┌─┐         
      link_qubit_1: ──────░───┼────┼──┤ X ├┤ X ├─░──╫──┤M├──|0>──░───┼────┼──┤ X ├┤ X ├─░──╫──┤M├──|0>──░───┼────┼──┤ X ├┤ X ├─░──╫─┤M├─────────
                    ┌───┐ ░   │    │  └─┬─┘└─┬─┘ ░  ║  └╥┘       ░   │    │  └─┬─┘└─┬─┘ ░  ║  └╥┘       ░   │    │  └─┬─┘└─┬─┘ ░  ║ └╥┘┌─┐      
      code_qubit_0: ┤ X ├─░───■────┼────┼────┼───░──╫───╫────────░───■────┼────┼────┼───░──╫───╫────────░───■────┼────┼────┼───░──╫──╫─┤M├──────
                    ├───┤ ░        │    │    │   ░  ║   ║        ░        │    │    │   ░  ║   ║        ░        │    │    │   ░  ║  ║ └╥┘┌─┐   
      code_qubit_1: ┤ X ├─░────────■────■────┼───░──╫───╫────────░────────■────■────┼───░──╫───╫────────░────────■────■────┼───░──╫──╫──╫─┤M├───
                    ├───┤ ░                  │   ░  ║   ║        ░                  │   ░  ║   ║        ░                  │   ░  ║  ║  ║ └╥┘┌─┐
      code_qubit_2: ┤ X ├─░──────────────────■───░──╫───╫────────░──────────────────■───░──╫───╫────────░──────────────────■───░──╫──╫──╫──╫─┤M├
                    └───┘ ░                      ░  ║   ║        ░                      ░  ║   ║        ░                      ░  ║  ║  ║  ║ └╥┘
round_0_link_bit: 2/════════════════════════════════╩═══╩══════════════════════════════════╬═══╬══════════════════════════════════╬══╬══╬══╬══╬═
                                                    0   1                                  ║   ║                                  ║  ║  ║  ║  ║ 
round_1_link_bit: 2/═══════════════════════════════════════════════════════════════════════╩═══╩══════════════════════════════════╬══╬══╬══╬══╬═
                                                                                           0   1                                  ║  ║  ║  ║  ║ 
round_2_link_bit: 2/══════════════════════════════════════════════════════════════════════════════════════════════════════════════╩══╩══╬══╬══╬═
                                                                                                                                  0  1  ║  ║  ║ 
        code_bit: 3/════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╩══╩══╩═
                                                                                                                                        0  1  2

In [29]:

initial_layout = [1, 5, 0, 3, 6] # for link_qubit and code_qubit [link_qubit, code_qubit]
transpiled_qc = transpile(qc, backend, initial_layout=initial_layout) #* virtual to physical: [0, 3, 5]  
job_id = backend.run(transpiled_qc, shots=1000, meas_level=1, meas_return='single').job_id()  # meas_level 1: IQ (0 is counts), meas_return: single for info from every shot (not 'avg')
job_id


'cms6mn2kvec0008wgfh0'

In [30]:
job = provider.retrieve_job(job_id)
job.status()

<JobStatus.RUNNING: 'job is actively running'>

In [31]:
memory = job.result().get_memory()
np.shape(memory) 
counts = get_counts(memory)
counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
counts

[('100000111', 351),
 ('101010011', 45),
 ('100011101', 44),
 ('100010011', 42),
 ('100001110', 39),
 ('100000011', 32),
 ('101111101', 27),
 ('100101110', 26),
 ('101000111', 23),
 ('100000110', 18),
 ('100100111', 16),
 ('100111101', 12),
 ('000000111', 12),
 ('100100110', 11),
 ('100001111', 11),
 ('110000111', 11),
 ('101101001', 9),
 ('001010011', 7),
 ('100010111', 7),
 ('101111100', 7),
 ('101010111', 7),
 ('100010010', 7),
 ('100101001', 7),
 ('110101110', 7),
 ('100001101', 6),
 ('101000011', 6),
 ('100111010', 6),
 ('100000101', 5),
 ('101011010', 5),
 ('101110100', 5),
 ('101011101', 5),
 ('100010001', 4),
 ('101111001', 4),
 ('101110101', 4),
 ('100010100', 4),
 ('100001010', 4),
 ('100011001', 4),
 ('110001110', 4),
 ('100011010', 4),
 ('100101111', 4),
 ('101111010', 4),
 ('101001001', 3),
 ('011111101', 3),
 ('100101010', 3),
 ('101100100', 3),
 ('101101101', 3),
 ('001000011', 3),
 ('101001101', 3),
 ('000010111', 3),
 ('110111010', 3),
 ('110100110', 3),
 ('001111101',